# Problema prototipo de cuarto orden

#### Alejandro Alonso Membrilla y Pilar Navarro Ramírez

In [1]:
import numpy as np # Importamos el módulo NumPy con el pseudónimo np
import sympy as sp # Importamos el módulo SymPy con el pseudónimo sp
import math
from sympy import sin, cos
%matplotlib inline 
import matplotlib.pyplot as plt  # plt será el pseudónimo correspondiente

t,z,xL,xR,yL,yR = sp.symbols('t,z,xL,xR,yL,yR')

In [2]:
# Base de elementos finitos de Hermite
def lh1(t):
    """primera func. de base del E.F. de Hermite P3 en 1D"""
    return (1+2*t) * (t-1)**2

def lh2(t):
    """segunda func. de base del E.F. de Hermite P3 en 1D"""
    return t * (t-1)**2

def lh3(t):
    """tercera func. de base del E.F. de Hermite P3 en 1D"""
    return -t**2 * (2*t-3)

def lh4(t):
    """cuarta func. de base del E.F. de Hermite P3 en 1D"""
    return t**2 * (t-1)

# def dlh1(t):
#     return 2*(t - 1)**2 + (2*t - 2)*(2*t + 1)

# def dlh2(t):
#     return t*(2*t - 2) + (t - 1)**2

# def dlh3(t):
#     return -2*t**2 - 2*t*(2*t - 3)

# def dlh4(t):
#     return t**2 + 2*t*(t - 1)

def Finv(z,a,b):
    """afinidad entre cada subintervalo de la partición 
        y el intervalo unidad"""
    return (z-a)/(b-a)

def dFinv(z,a,b):
    """ derivada de la afinidad Finv"""
    return 1/(b-a)

In [4]:
def wi0(z,x,i):
    """funciones de base del E.F. de Hermite P3 unidimensional 
    que controlan el valor de la función"""
    if (x[i-1]<=z)*(z<=x[i]):
        valor = lh1(Finv(z,x[i-1],x[i]))
    elif (x[i]<=z)*(z<=x[i+1]):
        valor = lh3(Finv(z,x[i-1],x[i]))
    return valor

def wi1(z,x,i):
    """funciones de base del E.F. de Hermite P3 unidimensional
    que controlan la derivada"""
    if (x[i-1]<=z)*(z<=x[i]):
        valor = lh2(Finv(z,x[i-1],x[i]))*(x[i]-x[i-1])
    elif (x[i]<=z)*(z<=x[i+1]):
        valor = lh4(Finv(z,x[i-1],x[i]))*(x[i]-x[i-1])
    return valor

def w00(z,x,i):
    """función de base del E.F. de Hermite P3 unidimensional 
    para el valor de la función en el extremo izquierdo"""
    if (x[0]<=z)*(z<=x[1]):
        return lh3(Finv(z,x[i-1],x[i]))
    else:
        return 0

def w01(z,x,i):
    """función de base del E.F. de Hermite P3 unidimensional 
    para la derivada de la función en el extremo izquierdo"""
    if (x[0]<=z)*(z<=x[1]):
        return lh4(Finv(z,x[i-1],x[i]))*(x[i]-x[i-1])
    else:
        return 0

def wn0(z,x,i):
    """funciones de base del E.F. de Hermite P3 unidimensional
    para el valor de la función en el extremo derecho"""
    if (x[i-1]<=z)*(z<=x[i]):
        lh1(Finv(z,x[i-1],x[i]))
    else:
        return 0
    
def wn1(z,x,i):
    """funciones de base del E.F. de Hermite P3 unidimensional
    para la derivada de la función en el extremo derecho"""
    if (x[i-1]<=z)*(z<=x[i]):
        lh2(Finv(z,x[i-1],x[i]))
    else:
        return 0

In [6]:
a = 0;   b = 10; 

nx= 10;      x = np.linspace(a,b,nx+1);
nxx = 1000;  xx = np.linspace(a,b,nxx);            

## Ejercicio 2. Matriz de rigidez

In [15]:
func_base = [lh1,lh2,lh3,lh4]
Rk = [ 
        [ sp.integrate(sp.diff(p_i(Finv(z,xL,xR)),z,2)*sp.diff(p_j(Finv(z,xL,xR)),z,2),[z,xL,xR]) 
         for p_j in func_base ]
        for p_i in func_base
    ]

Rk = sp.simplify( sp.Matrix(Rk) )
Rk.subs({xL:0,xR:sp.Symbol('h_k')})

Matrix([
[ 12/h_k**3,  6/h_k**3, -12/h_k**3,  6/h_k**3],
[  6/h_k**3,  4/h_k**3,  -6/h_k**3,  2/h_k**3],
[-12/h_k**3, -6/h_k**3,  12/h_k**3, -6/h_k**3],
[  6/h_k**3,  2/h_k**3,  -6/h_k**3,  4/h_k**3]])

## Ejercicio 3. Vector de cargas

In [20]:
C = sp.Symbol('C')
dk = sp.simplify(sp.Matrix([ C*sp.integrate(p_i(Finv(z,xL,xR)),[z,xL,xR]) 
                             for p_i in func_base]
                          ))
dk.subs({xL:0,xR:sp.Symbol('h_k')})

Matrix([
[  C*h_k/2],
[ C*h_k/12],
[  C*h_k/2],
[-C*h_k/12]])